In [1]:
from src.utils import *

In [2]:
# delete 75% of files in the directory
import os
import random

def delete_files(directory, percentage=0.75):
    files = os.listdir(directory)
    files = random.sample(files, int(len(files) * percentage))
    for file in files:
        os.remove(os.path.join(directory, file))

# delete_files('data/train/audio_yes_no/no', 0.5)
# delete_files('data/train/audio_yes_no/yes', 0.5)

In [3]:
import torch
from transformers import AutoFeatureExtractor, Wav2Vec2ForSequenceClassification

model_name = 'superb/wav2vec2-base-superb-ks'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

In [4]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
import os

class CustomAudioDataset(Dataset):
    def __init__(self, data_dir, transform=None, fixed_length=None):
        self.data_dir = data_dir
        self.file_list, self.labels = self._get_file_list_and_labels()
        self.transform = transform
        self.fixed_length = fixed_length

    def _get_file_list_and_labels(self):
        file_list = []
        labels = []
        for root, dirs, files in os.walk(self.data_dir):
            for file in files:
                if file.endswith(".wav"):  # Adjust file extension if needed
                    file_list.append(root + "/" + file)
                    labels.append(os.path.basename(root))  # Extract label from directory name
        return file_list, labels

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        waveform, sample_rate = torchaudio.load(file_path)
        
        
        
        if self.fixed_length:
            waveform = self._pad_waveform(waveform, self.fixed_length)

        label = self.labels[idx]

        

        if self.transform:
            waveform = self.transform(waveform, sampling_rate=sample_rate)['input_values'][0]

        return waveform, sample_rate, label

    def _pad_waveform(self, waveform, target_length):
        length_diff = target_length - waveform.size(1)
        if length_diff > 0:
            padding = torch.zeros((1, length_diff))
            waveform = torch.cat([waveform, padding], dim=1)
        return waveform.squeeze(0)

# Example usage
data_dir = "data/train/audio_small/"
# data_dir = "data/train/audio_yes_no/"
transform = feature_extractor  # You can define transformations if needed
fixed_length = 16000  # Assuming you want to fix the length to 16000 samples
sampling_rate = fixed_length  # Assuming you want to fix the sampling rate to 16000 Hz



batch_size = 4

In [5]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def freeze_layers_except_last_n(model, n):
    # Get all parameters
    parameters = list(model.parameters())
    total_layers = len(parameters)

    # Freeze all layers except the last n
    for i, param in enumerate(parameters):
        if i < total_layers - n:
            param.requires_grad = False




In [6]:
num_epochs = 10
perc = 0.05

In [7]:
optimizers = ['adam', 'sgd']
learning_rates = [0.0001, 0.001, 0.01]

In [8]:
num_features = 11

In [9]:
import random
import numpy as np

for opt in tqdm(optimizers, desc='Optimizers'):
    for lr in tqdm(learning_rates, desc="Learninig Rates"):
        if opt == 'adam':
            continue
        if opt == 'sgd' and lr == 0.0001:
            continue

        for i in tqdm(np.arange (0, 5, 1), desc='Training loop (5 times)'):
        
            random.seed(int(i))
            torch.manual_seed(i)
            torch.cuda.manual_seed(i)
            torch.cuda.manual_seed_all(i)

            
        
            model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)

            only_name = model_name.split("/")[-1]   

            if opt == 'adam':
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            else:
                optimizer = torch.optim.SGD(model.parameters(), lr=lr)


          
            criterion = torch.nn.CrossEntropyLoss()

            device = torch.device('cuda')
            # Assuming your model is named ast_model
            freeze_layers_except_last_n(model.wav2vec2.encoder.layers, 4)

            # set number of features to 11
            model.classifier = torch.nn.Linear(in_features=256, out_features=num_features, bias=True)

            model.to(device)

            train_dataset = CustomAudioDataset(data_dir, fixed_length=16000, transform=feature_extractor)
                                            


            n_train = len(train_dataset)
            n_val = int(perc * n_train)
            n_test = n_val//2
            n_train = n_train - n_val

            train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [n_train, n_val], generator=torch.Generator().manual_seed(42))
            val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [n_val-n_test, n_test], generator=torch.Generator().manual_seed(42))

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            

            labels = set(train_dataset.dataset.labels)
            label_to_index = dict((label, i) for i, label in enumerate(sorted(labels)))
            name = data_dir.split("/")[-2]
            # print(name)
            log_dir = train(model, train_loader, val_loader, num_epochs, optimizer, criterion, device, label_to_index, only_name, log=True, description=f"test_{name}_opt_{opt}_lr_{lr}_{i}")
            test(model, test_loader, criterion, device, label_to_index, only_name, log_dir)



Optimizers:   0%|          | 0/2 [00:00<?, ?it/s]

Learninig Rates:   0%|          | 0/3 [00:00<?, ?it/s]

Learninig Rates:   0%|          | 0/3 [00:00<?, ?it/s]

Training loop (5 times):   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\adamm\miniconda3\envs\torchenv\lib\site-packages\transformers\configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at superb/wav2vec2-base-superb-ks were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a mode

...

Epoch 10/10, Train Loss: 0.09339505726700272


Validation:   0%|          | 0/163 [00:00<?, ?it/s]

Epoch 10/10, Validation Loss: 0.1768353427505533, Validation Accuracy: 0.9477726574500768


Testing:   0%|          | 0/163 [00:00<?, ?it/s]

{'val_acc': [0.7542242703533026, 0.8509984639016898, 0.9078341013824884, 0.9293394777265745, 0.9231950844854071, 0.9324116743471582, 0.9216589861751152, 0.9539170506912442, 0.9447004608294931, 0.9477726574500768], 'val_loss': [0.836544535825589, 0.48882744003064427, 0.2984955173408601, 0.25512607815672583, 0.22179826525142815, 0.2040556421756253, 0.23042026336208069, 0.1654894144643538, 0.20509797450001643, 0.1768353427505533], 'train_loss': [1.469098839897398, 0.5773969325776622, 0.4164318722438387, 0.26806777939709564, 0.22861579519602868, 0.1820723991180679, 0.15959525167990377, 0.12132609457243168, 0.0950484327900668, 0.09339505726700272], 'test_correct_in_batch': [4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 2.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 2.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.